In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import image, masking, input_data

In [ ]:

base_dir = '/Volumes/Li/task-persuade/braindata'
denoised = 'denoised 5'

fwhm=6
tr = 2

tmp = 'Schaefer'

Schaefer_mask_file = '/Users/li/Desktop/template/Schaefer/tpl-MNI152NLin2009cAsym_res-02_atlas-Schaefer2018_desc-200Parcels7Networks_dseg.nii.gz'
roi_label_file = '/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv'
Schaefer_mask = nib.load(Schaefer_mask_file)
roi_label = pd.read_csv(roi_label_file)['ROI Name'].values.tolist()

brain_mask_path = '/Users/li/Desktop/template/brain mask/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii.gz'

sub_list = [f'sub-{x:0>2d}' for x in range(1,36)]
run_list = [1,2,3,4,5]

for sub in sub_list:
    new_folder_path = os.path.join(base_dir, denoised, 'denoised', sub)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    for run in run_list:
        
        run_file = f'/Volumes/Li/task-persuade/fmriprep/{sub}/func/{sub}_task-persuade_run-{run}_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'
        run_img = nib.load(run_file)
        
        confound_file = f'/Volumes/Li/task-persuade/fmriprep/{sub}/func/{sub}_task-persuade_run-{run}_desc-confounds_timeseries.tsv'
        confound_df = pd.read_csv(confound_file, delimiter ='\t')
        
        selected_confound_cols = ['csf','white_matter','global_signal',
                                  'trans_x','trans_y','trans_z','rot_x','rot_y','rot_z',
                                  'trans_x_derivative1','trans_y_derivative1','trans_z_derivative1',
                                  'rot_x_derivative1','rot_y_derivative1','rot_z_derivative1',
                                  # 'trans_x_power2','trans_y_power2','trans_z_power2',
                                  # 'rot_x_power2','rot_y_power2','rot_z_power2',
                                  # 'trans_x_derivative1_power2','trans_y_derivative1_power2','trans_z_derivative1_power2',
                                  # 'rot_x_derivative1_power2','rot_y_derivative1_power2','rot_z_derivative1_power2'
                                 ]
        
        confound_selected = confound_df[selected_confound_cols]
        confound_selected = confound_selected.fillna(value=0)
        confound_matrix = confound_selected.values

        denoised_img = image.clean_img(run_img, confounds=confound_matrix,
                                       detrend=True, standardize=True, high_pass=0.008, t_r=tr, mask_img=brain_mask_path)
        
        smoothed_img = image.smooth_img(denoised_img, fwhm=fwhm)
        
        save_path = os.path.join(base_dir, denoised, 'denoised', sub, f'{sub}_task-persuade_run-{run}_denoised_no_smooth_space-MNI152NLin2009cAsym_res-2_bold.nii.gz')
        nib.save(smoothed_img,save_path)
        # print(f'{sub} {run} denoised_smoothed data saved successfully!')

    print(f'{sub} Done!')
    print('-----------------------------------------------')



In [ ]:

for sub in sub_list:
 
    new_folder_path = os.path.join(base_dir, denoised, 'parcel data', tmp, sub)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    for run in run_list:

        run_file = f'/Volumes/Li/task-persuade/braindata/{denoised}/denoised/{sub}/{sub}_task-persuade_run-{run}_denoised_no_smooth_space-MNI152NLin2009cAsym_res-2_bold.nii.gz'
        run_image = nib.load(run_file)
        
        masker = input_data.NiftiLabelsMasker(labels_img = Schaefer_mask_file, standardize=True)
        
        parcellations_time_series = masker.fit_transform(run_image)
        
        parcellations_time_series_df = pd.DataFrame(parcellations_time_series) # , columns=roi_label
        
        
        csv_file_path = f'/Volumes/Li/task-persuade/braindata/{denoised}/parcel data/{tmp}/{sub}/{sub}_run-{run}_task-persuade_{tmp}_time-series.csv'
        parcellations_time_series_df.to_csv(csv_file_path, index=False)

    print(f'{sub} Done!')



## note: The presentation order of experimental stimuli was counterbalanced using a Latin square design. 
## Subject 20 originally scheduled the session for the second day but missed it due to his oversleeping. He completed the experiment on the third day, following the stimulus order originally assigned for the second day.

In [ ]:

new_folder_path = os.path.join(base_dir, denoised, 'parcel data', 'Schaefer align 5 runs')
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

import itertools

# sub-20 安排的是第二天，但睡过头了，没来，第三天另外加上的，但仍然按第二天安排的顺序做实验
sub_lists = [['01','02','03','04','05','06','07'],
             ['08','09',10,11,12,13,20],
             [14,15,16,17,18,19,21],
             [22,23,24,25,26,27,28],
             [29,30,31,32,33,34,35]]

# day1的run1对应day2的run5
run_lists = [[1,2,3,4,5],
            [5,1,2,3,4],
            [4,5,1,2,3],
            [3,4,5,1,2],
            [2,3,4,5,1]]


# time points
time_lists = [[10,176,48,152,48,140,30],
 [10,134,48,162,48,168,30],
 [10,166,48,124,48,182,30],
 [10,138,48,132,48,188,30],
 [10,170,48,150,48,148,30]]

# 计算累加时间,转成TR数
tr_lists = [[int(x/2) for x in itertools.accumulate(sublist)] for sublist in time_lists]


# 把15个片段的数据拼接起来

subs_data = []

for day in list(range(5)):
    
    for sub in sub_lists[day]:
        sub_data = []
        
        for index,run in enumerate(run_lists[day]):
            
            tr = tr_lists[index]
            file = f'/Volumes/Li/task-persuade/braindata/{denoised}/parcel data/{tmp}/sub-{sub}/sub-{sub}_run-{run}_task-persuade_{tmp}_time-series.csv'
            df = pd.read_csv(file)
            
            # 分别提取三段视频的BOLD
            run_data = np.array(pd.concat([df.iloc[tr[0]:tr[1]], 
                                           df.iloc[tr[2]:tr[3]], 
                                           df.iloc[tr[4]+1:tr[5]+1]]))   # For the third video in each run, the onset time was manually adjusted due to a delay in the program.
            sub_data.append(run_data)
        sub_data = np.vstack(sub_data)
        subs_data.append(sub_data)
        
subs_data = np.array(subs_data)
print(subs_data.shape)

np.save(f'/Volumes/Li/task-persuade/braindata/{denoised}/parcel data/Schaefer align 5 runs/subs_data.npy', subs_data)

